    def plot_scores(self):
        """
        This function aims at plotting the scores sampled by the environment
        :return: None
        """
        # set limits for the plot
        plt.xlim(self.horizon)
        plt.ylim(max(max(self.scores.values())) + 1)
        plt.figure(figsize=(20, 20))

        for key in self.scores:
            x = np.arange(0, self.pulls[key], 1)
            plt.plot(x, self.scores[key], "o-", label=str(key))

        # set all the stuff in the plot
        plt.title("Bandits Behavior T=" + str(self.horizon))
        plt.xlabel("Pulls")
        plt.ylabel("Reward")
        plt.legend(loc="best")
        plt.grid()

        # save the plot
        now = datetime.now()
        current_time = str(now.hour) + '_' + str(now.minute) + '_' + str(now.second)
        current_date = str(now.year) + str(now.month) + str(now.day) + '__'
        plt.savefig(self.plot_path + '/' + 'Rising_Bandits_Behavior_' + str(self.sim_counter) + '_' + current_date +
                    current_time + '.pdf',
                    format='pdf')
        plt.clf()

        return

    def plot_projections(self):
        """
        This function aims at plotting the scores and the projection of the final value
        :return: None
        """
        assert self.n_arms > 1, "Error, need more arms"
        fig, axs = plt.subplots(self.n_arms, figsize=(20, 20))

        x_max = self.horizon
        y_max = max(max(self.projections.values())) + 1

        i = 0
        for key in self.scores:
            axs[i].set_xlim(0, x_max)
            axs[i].set_ylim(0, y_max)

            for j in range(int(self.pulls[key])):
                if self.projections[key][j] != 0:
                    x = [j, self.horizon]
                    y = [self.scores[key][j], self.projections[key][j]]
                    axs[i].plot(x, y, '-.k', alpha=.3)

            x = np.arange(0, len(self.scores[key]), 1)
            y = self.scores[key]
            axs[i].plot(x, y, 'o-', label=str(key))

            x = [self.horizon] * len(self.projections[key])
            y = self.projections[key]
            axs[i].plot(x, y, 'o')

            axs[i].set_xlabel('Time')
            axs[i].set_ylabel('Scores')
            axs[i].legend(loc='best')
            axs[i].grid()

            i += 1

        # save the plot
        now = datetime.now()
        current_time = str(now.hour) + '_' + str(now.minute) + '_' + str(now.second)
        current_date = str(now.year) + str(now.month) + str(now.day) + '__'
        plt.savefig(self.plot_path + '/' + 'Projection_Behavior_' + str(self.sim_counter) + '_' + current_date +
                    current_time + '.pdf',
                    format='pdf')
        plt.clf()
        return

    def plot_bounds(self):
        assert self.n_arms > 1, "Error, need more arms"
        fig, axs = plt.subplots(self.n_arms, figsize=(20, 20))

        x_max = int(max(self.pulls)) + 1
        y_max = max(max(self.ubs.values())) + .5
        y_min = min(0, min(min(self.lbs.values())) - .5)

        i = 0
        for key in self.scores:
            axs[i].set_xlim(0, x_max)
            axs[i].set_ylim(y_min, y_max)

            x = np.arange(0, int(self.pulls[key]), 1)
            y = self.scores[key]
            axs[i].plot(x, y, 'o-', label=str(key))

            where = [1] * int(self.pulls[key])
            for j in range(int(self.pulls[key])):
                if self.ubs[key][j] == 0:
                    where[j] = 0

            y_up = self.ubs[key]
            axs[i].fill_between(x, y, y_up, alpha=.3, where=where)

            y_down = self.lbs[key]
            axs[i].fill_between(x, y, y_down, alpha=.3, where=where)

            axs[i].set_xlabel('Time')
            axs[i].set_ylabel('Scores')
            axs[i].legend(loc='best')
            axs[i].grid()
            i += 1

        # save the plot
        now = datetime.now()
        current_time = str(now.hour) + '_' + str(now.minute) + '_' + str(now.second)
        current_date = str(now.year) + str(now.month) + str(now.day) + '__'
        plt.savefig(self.plot_path + '/' + 'Bounds_Behavior_' + str(self.sim_counter) + '_' + current_date +
                    current_time + '.pdf',
                    format='pdf')
        plt.clf()

        return

    def plot_pulls(self):
        """
        This function aims at plotting the pulls in groups, for all the bandits
        :return: None
        """
        # check the number of groups to show
        assert 0 < self.group_num <= self.horizon, "Error in group_num"
        assert self.horizon % self.group_num == 0, "Number of groups not compliant with the horizon"

        # width of the bar
        width = .35 / self.n_arms
        off = []
        mid = math.floor(self.n_arms / 2)
        for i in range(self.n_arms):
            off.append(i - mid)

        # how many groups
        x = np.arange(self.group_num)
        labels = list(self.scores.keys())

        # create the subplots
        fig, ax = plt.subplots(figsize=(20, 20))
        for key in self.group_pulls:
            rect = ax.bar(x + off[int(key)] * width, self.group_pulls[key], width, label=str(key))
            ax.bar_label(rect, padding=3)

        # set the names
        ax.set_title("Pulls by group of " + str(self.horizon / self.group_num) + " T=" + str(self.horizon))
        ax.set_ylabel("Pulls")
        ax.set_xlabel("Groups of " + str(self.horizon / self.group_num) + " Pulls")
        ax.set_xticks(x, labels)
        ax.grid()
        ax.legend(loc='best')
        fig.tight_layout()

        # save the plot
        now = datetime.now()
        current_time = str(now.hour) + '_' + str(now.minute) + '_' + str(now.second)
        current_date = str(now.year) + str(now.month) + str(now.day) + '__'
        plt.savefig(self.plot_path + '/' + 'Pulls_Behavior_' + str(self.sim_counter) + '_' + current_date +
                    current_time + '.pdf',
                    format='pdf')
        plt.clf()

        return

    def _sr_plot_scores(self):
        # set limits for the plot
        plt.xlim(self.horizon)
        plt.ylim(max(max(self.scores.values())) + 1)
        plt.figure(figsize=(20, 20))

        for key in self.scores:
            x = np.arange(0, self.pulls[key], 1)
            plt.plot(x, self.scores[key], "o-", label=str(key))

        # set all the stuff in the plot
        plt.title("Bandits Behavior T = " + str(self.horizon))
        plt.xlabel("Pulls")
        plt.ylabel("Reward")
        plt.legend(loc="best")
        plt.grid()

        # save the plot
        now = datetime.now()
        current_time = str(now.hour) + '_' + str(now.minute) + '_' + str(now.second)
        current_date = str(now.year) + str(now.month) + str(now.day) + '__'
        plt.savefig(self.plot_path + '/' + 'SR_Rising_Bandits_Behavior_' + str(self.sim_counter) + '_' + current_date +
                    current_time + '.pdf',
                    format='pdf')
        plt.clf()
        return